# Exercise 3

Welcome to the third challenge! The goal of this exercise is to combine the two agents from exercise 1 and 2 into a multi-agent system. 

## Setup
Let's start by importing the libraries that we will need.

In [ ]:
import os
import re
from datetime import datetime
from typing import Optional

import requests
from crewai import Agent, Crew, Task
from crewai_tools import BaseTool, tool
from dotenv import load_dotenv
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar

Before you execute this cell, make sure to provide the environment variables `OPENAI_API_BASE`, `OPENAI_MODEL_NAME`, and `OPENAI_API_KEY` in the `.env` file.

In [ ]:
load_dotenv(override=True)

assert "OPENAI_MODEL_NAME" in os.environ, "No model specified in .env file!"
print("Using the following LLM model:", os.environ.get("OPENAI_MODEL_NAME"))

In [ ]:
gc = GoogleCalendar("agents.wt24@gmail.com", credentials_path="../credentials/credentials.json")

## Tools

In [ ]:
# Used for the GetCurrentDateAndTimeTool
weekday_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}


# Helper function
def format_duration(duration: str) -> str:
    """Formats the duration returned by the public transport API such that it is more readable."""
    readable_duration_str = "Invalid format"

    match = re.match(r"(\d{2})d(\d{2}):(\d{2}):(\d{2})", duration)
    if match:
        days, hours, minutes, seconds = map(int, match.groups())

        readable_duration = []

        if days > 0:
            readable_duration.append(f"{days} day{'s' if days > 1 else ''}")
        if hours > 0:
            readable_duration.append(f"{hours} hour{'s' if hours > 1 else ''}")
        if minutes > 0:
            readable_duration.append(f"{minutes} minute{'s' if minutes > 1 else ''}")
        if seconds > 0:
            readable_duration.append(f"{seconds} second{'s' if seconds > 1 else ''}")

        readable_duration_str = ", ".join(readable_duration)

    return readable_duration_str


class GetEventsTool(BaseTool):
    name: str = "Get events within a certain time span from the calendar"
    description: str = (
        "Returns a list of dictionaries that store information about the events in the specified time span."
    )

    def _run(self, start_datetime_isoformat: str, end_datetime_isoformat: str) -> dict:
        start = datetime.fromisoformat(start_datetime_isoformat)
        end = datetime.fromisoformat(end_datetime_isoformat)

        events = gc.get_events(time_min=start, time_max=end)
        output = []

        for event in events:
            recurrence = event.recurrence[0] if event.recurrence else None
            output.append(
                {
                    "title": event.summary,
                    "description": event.description,
                    "location": event.location,
                    "start": event.start.isoformat(),
                    "end": event.end.isoformat(),
                    "recurrence": recurrence,
                }
            )

        return output

    def cache_function(*args):
        return False


class CreateEventTool(BaseTool):
    name: str = "Creates a new event in the calendar."
    description: str = "This tool can be used to create new events in the calendar."

    def _run(
        self,
        start_datetime_isoformat: str,
        end_datetime_isoformat: str,
        event_title: str,
        event_description: str,
        event_location: Optional[str] = None,
    ) -> None:
        start = datetime.fromisoformat(start_datetime_isoformat)
        end = datetime.fromisoformat(end_datetime_isoformat)

        event = Event(
            summary=event_title, description=event_description, start=start, end=end, location=event_location
        )

        gc.add_event(event)

    def cache_function(*args):
        return False


class GetCurrentDateAndTimeTool(BaseTool):
    name: str = "Get the current date and time"
    description: str = "Returns a dictionary with information about the current date and time."

    def _run(self) -> dict:
        now = datetime.now()
        return {
            "year": now.year,
            "month": now.month,
            "day": now.day,
            "hour": now.hour,
            "minute": now.minute,
            "weekday": weekday_map[now.weekday()],
        }

    def cache_function(*args):
        return False


@tool("Get public transport connections")
def get_connections(start: str, end: str, date: str, time: str, is_arrival_time: bool):
    """
    Gets public transport connections for a given start and end location and a specific date and time.
    Requires UTF-8 encoded arguments, do not use unicode characters!

    :param start: A string representing either the name of the station or its ID
    :param end: A string representing either the name of the station or its ID
    :param date: The date for which to check the connections (iso format)
    :param time: The time for which to check the connections (%H:%M)
    :param is_arrival_time: Boolean value specifying whether the date and time refer
        to the arrival (True) or the departure (False). The argument should be formatted
        as a string because it will be converted into a boolean upon executing this tool.
    """
    response = requests.get(
        f"http://transport.opendata.ch/v1/connections?from={start}&to={end}&date={date}&time={time}&isArrivalTime={int(is_arrival_time)}"
    )
    data = response.json()

    connections = []
    for connection in data["connections"]:
        departure = datetime.strptime(connection["from"]["departure"], "%Y-%m-%dT%H:%M:%S%z")
        arrival = datetime.strptime(connection["to"]["arrival"], "%Y-%m-%dT%H:%M:%S%z")

        connections.append(
            {
                "from": connection["from"]["station"]["name"],
                "departure_platform": connection["from"]["platform"],
                "departure_date": {"year": departure.year, "month": departure.month, "day": departure.day},
                "departure_time": {"hour": departure.hour, "minute": departure.minute},
                "departure_delay": connection["from"]["delay"],
                "to": connection["to"]["station"]["name"],
                "arrival_platform": connection["to"]["platform"],
                "arrival_date": {"year": arrival.year, "month": arrival.month, "day": arrival.day},
                "arrival_time": {"hour": arrival.hour, "minute": arrival.minute},
                "arrival_delay": connection["to"]["delay"],
                "duration": format_duration(connection["duration"]),
            }
        )

    if not connections:
        raise Exception(
            "Couldn't find any connection, please verify that all of the arguments are correctly formatted in UTF-8"
        )

    return connections

## Agents

In [ ]:
event_manager = Agent(
    role="Senior Event Manager",
    goal=(
        "Use your event management expertise to help the user "
        "with his request. Make sure that you incorporate your "
        "domain knowldege. For example, you are aware that even "
        "if an event in a calendar is an all-day event, it does "
        "not always mean that the user is really blocked by this "
        "event the entire day. Make sure to infer as much as possible "
        "from the event information that you have access to. Here is "
        "the user's request: {user_request}"
    ),
    backstory="An knowledgeable expert in planning and managing events.",
    tools=[GetCurrentDateAndTimeTool(), GetEventsTool(), CreateEventTool()],
    cache=False,
    verbose=True,
)

public_transport_agent = Agent(
    role="Public Transport Specialist",
    goal=(
        "Use your planning expertise to look up public transport information "
        "to answer the user's question. Make sure to plan everything such that "
        "it results in the best possible option for the user. Waiting times "
        "should be minimized and the user should always be on time in case "
        "a specific departure or arrival time is specified by the user. "
        "Here is the question of the user: {user_request}"
    ),
    backstory="A knowledgeable expert in planning trips involving public transport.",
    tools=[GetCurrentDateAndTimeTool(), get_connections],
    verbose=True,
)

## Tasks

In [ ]:
calendar_task = Task(
    description="Handle the user's request regarding his calendar. The language of your answer should match the language of the question.",
    expected_output="A concise and helpful answer to the user's request.",
    agent=event_manager,
)

public_transport_task = Task(
    description="Look up public transport information to answer the user's questions. The language of your answer should match the language of the question.",
    expected_output="A concise and helpful answer to the user's query.",
    agent=public_transport_agent,
)

## Kicking off the Crew

Inspiration for user request:

- Ich wohne in Bern und muss am Mittwoch um spätestens 9 Uhr in Suurstoffi 1, 6343 Rotkreuz sein. Bitte suche mir eine entsprechende Verbindung heraus und trage diese in meinen Kalender ein.

In [ ]:
crew = Crew(agents=[public_transport_agent, event_manager], tasks=[public_transport_task, calendar_task], cache=False)

request = input("Anliegen: ")

result = crew.kickoff(inputs={
    "user_request": request
})

print("Reply:", result.raw)